In [1]:
import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from preprocess_utility import *
from data_manager import *
from metadata import *

Setting environment for AWS compute node


No vtk


In [2]:
paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', 'SNC', 'SNR', '3N', '4N',
                    'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
structures = paired_structures + singular_structures

def wait_qsub_complete():
    op = "runall.sh"
    while "runall.sh" in op:
        op=subprocess.check_output('qstat')
        time.sleep(5)
        
def set_asg_cap(asg, desired_cap):
    num_hosts = subprocess.check_output('qhost').count('\n') - 3
    if num_hosts < desired_cap:
            subprocess.call("aws autoscaling set-desired-capacity --auto-scaling-group-name "+asg+" --desired-capacity "+ str(desired_cap), shell=True) 
            print("Scaling cluster:"+asg+" Size:"+ str(desired_cap))
            time.sleep(240)
            
all_stacks = ['MD589']

In [6]:
exclude_nodes = [33, 31]

# Generate score volumes

In [3]:
train_sample_scheme = 1

In [13]:
set_asg_cap("cfncluster-mbacluster-ComputeFleet-MSG4A54YHGPC", len(all_stacks))
for stack in all_stacks:
# for stack in ['MD635']:
    
    first_sec, last_sec = metadata_cache['section_limits'][stack]
    
#     #################################

    t = time.time()
    sys.stderr.write('running svm classifier ...')


    run_distributed5(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(train_sample_scheme)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning') + '/svm_predict.py',
                    'stack': stack,
                    'train_sample_scheme': train_sample_scheme},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')
    
    wait_qsub_complete()
    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 220 seconds

# #     #################################

    t = time.time()
    sys.stderr.write('interpolating scoremaps ...')

    run_distributed5(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d %(train_sample_scheme)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning') + '/interpolate_scoremaps_v2.py',
                    'stack': stack,
                    'train_sample_scheme': train_sample_scheme},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')
    
    wait_qsub_complete()
    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 50s/section

    #################################

    t = time.time()
    sys.stderr.write('visualize scoremaps ...')

    add_annotation = False
    # first_sec, last_sec = DataManager.load_cropbox(stack)[4:]

    run_distributed5(command='%(script_path)s %(stack)s -b %%(first_sec)d -e %%(last_sec)d %(add_annotation)s -t %(train_sample_scheme)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning') + '/visualize_scoremaps_v2.py',
                    'stack': stack,
                     'train_sample_scheme': train_sample_scheme,
                    'add_annotation': '-a' if add_annotation else ''},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')

    wait_qsub_complete()
    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 800 seconds / stack


    #################################
    
    t = time.time()
    sys.stderr.write('constructing score volumes ...')

    run_distributed5(command='%(script_path)s %(stack)s %%(label)s %(train_sample_scheme)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'reconstruct') + '/construct_score_volume_v2.py',
                    'stack': stack,
                    'train_sample_scheme': train_sample_scheme},
                    kwargs_list=dict(label=structures),
                    exclude_nodes=exclude_nodes,
                    argument_type='single')

    wait_qsub_complete()
    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 600 seconds

    #################################

    downscale_factor = 32

    #################################

    print stack

    volume_allLabels = {}

    for name_u in structures:
        try:
            volume = DataManager.load_score_volume(stack, label=name_u, downscale=downscale_factor, train_sample_scheme=train_sample_scheme)
            volume_allLabels[name_u] = volume            
#         del volume
        except:
            sys.stderr.write('Score volume for %s does not exist.\n' % name_u)

    t1 = time.time()

    gradient_dir = create_if_not_exists(os.path.join(VOLUME_ROOTDIR, stack, 'score_volume_gradients'))

#     for name_u in structures:
    for name_u in volume_allLabels.keys():

        t = time.time()

        gy_gx_gz = np.gradient(volume_allLabels[name_u].astype(np.float32), 3, 3, 3) 
        # 3.3 second - re-computing is much faster than loading
        # .astype(np.float32) is important; 
        # Otherwise the score volume is type np.float16, np.gradient requires np.float32 and will have to convert which is very slow
        # 20s (float32) vs. 2s (float16)

        sys.stderr.write('Gradient %s: %f seconds\n' % (name_u, time.time() - t))

        t = time.time()

        bp.pack_ndarray_file(gy_gx_gz[0], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gy.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))
        bp.pack_ndarray_file(gy_gx_gz[1], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gx.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))
        bp.pack_ndarray_file(gy_gx_gz[2], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gz.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))

        del gy_gx_gz

        sys.stderr.write('save %s: %f seconds\n' % (name_u, time.time() - t))


    sys.stderr.write('overall: %f seconds\n' % (time.time() - t1))

Scaling cluster:cfncluster-mbacluster-ComputeFleet-MSG4A54YHGPC Size:1


running svm classifier ...

/shared/MouseBrainAtlas/learning/svm_predict.py MD589 347 370 1 


done in 175.976615 seconds
interpolating scoremaps ...

/shared/MouseBrainAtlas/learning/interpolate_scoremaps_v2.py MD589 347 370 1 


...

done in 165.906997 seconds
constructing score volumes ...

/shared/MouseBrainAtlas/reconstruct/construct_score_volume_v2.py MD589 IC 1


done in 15.099550 seconds


MD589


Gradient Tz: 1.537107 seconds
save Tz: 1.449317 seconds
Gradient VCA: 1.423544 seconds
save VCA: 2.561206 seconds
Gradient 7n: 1.407359 seconds
save 7n: 1.955908 seconds
Gradient DC: 1.431982 seconds
save DC: 3.325554 seconds
Gradient 5N: 1.539244 seconds
save 5N: 2.438446 seconds
Gradient 3N: 1.432235 seconds
save 3N: 3.097871 seconds
Gradient Pn: 1.548391 seconds
save Pn: 3.056241 seconds
Gradient 10N: 1.525342 seconds
save 10N: 1.314991 seconds
Gradient LC: 1.529409 seconds
save LC: 1.940578 seconds
Gradient 7N: 1.553185 seconds
save 7N: 3.018483 seconds
Gradient Amb: 1.545700 seconds
save Amb: 2.706278 seconds
Gradient 12N: 1.483591 seconds
save 12N: 2.086305 seconds
Gradient RMC: 1.570682 seconds
save RMC: 3.632709 seconds
Gradient Sp5O: 1.542242 seconds
save Sp5O: 3.786730 seconds
Gradient Sp5I: 1.537335 seconds
save Sp5I: 3.581259 seconds
Gradient Sp5C: 1.536965 seconds
save Sp5C: 3.509956 seconds
Gradient VCP: 1.544679 seconds
save VCP: 3.072454 seconds
Gradient AP: 1.535239 se

# Global align

In [17]:
# Align
set_asg_cap("cfncluster-mbacluster-ComputeFleet-MSG4A54YHGPC", len(all_stacks))
t = time.time()
sys.stderr.write('align all subjects to atlas ...')

run_distributed5(command='%(script_path)s %%(stack)s %(train_sample_scheme)d 1 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/align_subject_brain_to_atlas_v2.py',
                'train_sample_scheme': train_sample_scheme},
                 kwargs_list=dict(stack=all_stacks),
#                 kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

wait_qsub_complete()
sys.stderr.write('done in %f seconds\n' % (time.time() - t))  # 526 seconds

align all subjects to atlas ...

/shared/MouseBrainAtlas/registration/align_subject_brain_to_atlas_v2.py MD589 1 1 atlasV2


done in 503.243499 seconds


In [20]:
# Transform
set_asg_cap("cfncluster-mbacluster-ComputeFleet-MSG4A54YHGPC", len(all_stacks))
t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed5(command='%(script_path)s %%(stack)s %(train_sample_scheme)d 1 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/transform_brains.py',
                'train_sample_scheme': train_sample_scheme},
                 kwargs_list=dict(stack=all_stacks),
#                 kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

wait_qsub_complete()
sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 310 seconds

transform atlas ...

/shared/MouseBrainAtlas/registration/transform_brains.py MD589 1 1 0 atlasV2


done in 171.125217 seconds


In [8]:
# Visualize
set_asg_cap("cfncluster-mbacluster-ComputeFleet-MSG4A54YHGPC", len(all_stacks))
t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed5(command='%(script_path)s %%(stack)s %(train_sample_scheme)d 1 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/visualize_aligned_brains_v2.py',
                'train_sample_scheme': train_sample_scheme},
                 kwargs_list=dict(stack=all_stacks),
#                kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

wait_qsub_complete()
sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 625 seconds

visualize aligned atlas overlay ...

/shared/MouseBrainAtlas/registration/visualize_aligned_brains_v2.py MD589 1 1 0 atlasV2


done in 200.986250 seconds


# Local align

In [9]:
# Local align

set_asg_cap("cfncluster-mbacluster-ComputeFleet-MSG4A54YHGPC", len(all_stacks))
t = time.time()
sys.stderr.write('LOCAL align ...')

run_distributed5(command='%(script_path)s %%(stack)s %(train_sample_scheme)d 1 2 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/fit_atlas_structure_to_subject_v2.py',
                'train_sample_scheme': train_sample_scheme},
                 kwargs_list=dict(stack=all_stacks),
#                kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

wait_qsub_complete()
sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 6807 seconds

LOCAL align ...

/shared/MouseBrainAtlas/registration/fit_atlas_structure_to_subject_v2.py MD589 1 1 2 atlasV2


done in 4506.435871 seconds


In [27]:
# Transform

set_asg_cap("cfncluster-mbacluster-ComputeFleet-MSG4A54YHGPC", len(all_stacks))
t = time.time()
sys.stderr.write('transform atlas ...')

run_distributed5(command='%(script_path)s %%(stack)s %(train_sample_scheme)d 1 2 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/transform_brains_global_to_local.py',
                'train_sample_scheme': train_sample_scheme},
                 kwargs_list=dict(stack=all_stacks),
#                 kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')

wait_qsub_complete()
sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 310 seconds

transform atlas ...

['gcn-20-33.sdsc.edu', 'gcn-20-31.sdsc.edu'] are excluded
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 369.619889 seconds


In [28]:
# Visualize

set_asg_cap("cfncluster-mbacluster-ComputeFleet-MSG4A54YHGPC", len(all_stacks))
t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

run_distributed5(command='%(script_path)s %%(stack)s %(train_sample_scheme)d 1 2 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/visualize_aligned_brains_v2_local.py',
                'train_sample_scheme': train_sample_scheme},
                kwargs_list=dict(stack=['MD635']),
                exclude_nodes=exclude_nodes,
                argument_type='single')
wait_qsub_complete()
sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 190 seconds

visualize aligned atlas overlay ...

['gcn-20-33.sdsc.edu', 'gcn-20-31.sdsc.edu'] are excluded
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 647.941902 seconds


In [13]:
# Transform locally transformed volumes back to atlas space

t = time.time()
sys.stderr.write('transform atlas ...')

exclude_nodes = [33]

run_distributed5(command='%(script_path)s %%(stack)s 1 1 2 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/transform_brains_reverse_global.py'},
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 310 seconds

transform atlas ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 220.844733 seconds


# Snake

In [6]:
train_sample_scheme

8

In [7]:
# Graphcut

t = time.time()
sys.stderr.write('Graphcut ...')

exclude_nodes = [33]

# for stack in all_stacks:
for stack in ['MD635']:

    run_distributed4(command='%(script_path)s %(stack)s %%(name_sided)s %(train_sample_scheme)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/fit_contour_to_scoremap.py',
                    'stack': stack,
                    'train_sample_scheme': train_sample_scheme},
                    kwargs_list=dict(name_sided=[convert_to_left_name(s) for s in paired_structures] + \
                                     [convert_to_right_name(s) for s in paired_structures] + \
                                     [s for s in singular_structures]),
                    exclude_nodes=exclude_nodes,
                    argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

Graphcut ...

['gcn-20-33.sdsc.edu'] are excluded
gcn-20-31.sdsc.edu is down
Using nodes: ['gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 163.411004 seconds
